---
title: "Event-based vision: CNN vs SNN"

commentable: true

date: 2022-12-27
lastmod: 2022-12-27
draft: false

tags: ["SNN", "Event-based vision"]
summary: "Why a CNN using a variable frame rate does the job."
---

When recording data from event-based cameras, the camera normally timestamps events with a precision of microsecond resolution. This allows us to track extremely rapid motion such as a bullet shot from a gun or engine vibrations in the kHz range. A lot of day-to-day applications of event-based cameras however deal with things such as gesture recognition or other basic classification tasks, which record much slower movements. It is common practice to train a stateful spiking neural network to be able to integrate the information coming from the events which signify a change in illumination rather than an absolute grey-level value. But when training on GPUs, we always have to discretize time. So starting from microsecond resolution, how much of the information can you bin together until the network cannot distinguish it anymore?
I'm going to show you that instead of employing costly backprop-through-time (BPTT), which scales linearly with the amount of time steps, we can simply integrate some of the input upfront and feed it to a stateless CNN.
For that we're going to look at a classic benchmark, the IBM DVS gesture recognition task. It includes 11 hand gestures, some of which have clockwise/counter-clockwise which make it a bit harder for a stateless network to distinguish them.

## The dataset
![](https://research.ibm.com/interactive/dvsgesture/images/arm_roll.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/hand_clap.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/left_hand_clockwise.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/left_hand_counter_clockwise.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/left_hand_wave.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/right_hand_wave.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/right_hand_clockwise.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/right_hand_counter_clockwise.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/air_drums.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/air_guitar.gif)
![](https://research.ibm.com/interactive/dvsgesture/images/other_gesture.gif)